In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
import os
import json
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.model_selection import train_test_split
from sklearn.model_selection import PredefinedSplit, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
import sklearn
import torch
import torchtext
from torchtext.data import Field, LabelField
from torchtext.data import TabularDataset
from torchtext.data import Iterator, BucketIterator
import spacy
import en_core_web_sm
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
from tqdm import tqdm, trange
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, classification_report, confusion_matrix
import time
from collections import Counter
import tensorflow
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
from tqdm import tqdm, trange
import pandas as pd
import numpy as np
import io
import os
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, classification_report, confusion_matrix
import matplotlib
import matplotlib.pyplot as plt

path = "/content/drive/My Drive/Colab Notebooks/585_group/"

Using TensorFlow backend.


In [4]:
## Set seed of randomization and working device
manual_seed = 77
torch.manual_seed(manual_seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
n_gpu = torch.cuda.device_count()
if n_gpu > 0:
    torch.cuda.manual_seed(manual_seed)

print(torch.cuda.get_device_name(0))

cuda
Tesla K80


In [0]:
# reading in 100k data
df = pd.read_csv(path+"subset_100k.csv", index_col=0, encoding="utf-8").reset_index(drop=True)
# df_shuffled = df.sample(frac=1, random_state=123) #shuffle rows randomly
# df_shuffled = df_shuffled.drop(columns="index") #drops index to only keep text and label
train, validate, test = np.split(df.sample(frac=1, random_state=123).drop(columns="index"), 
                                                                          [int(.6*len(df)), int(.8*len(df))])
train_texts, train_labels = zip(*train.values) #resulting type is tuples
valid_texts, valid_labels = zip(*validate.values)
test_texts, test_labels = zip(*test.values)

In [6]:
#check distribution of shuffled data by class (random state 123)

print(Counter(train_labels))
print(Counter(valid_labels))
print(Counter(test_labels))

Counter({0: 30118, 1: 29882})
Counter({1: 10124, 0: 9876})
Counter({0: 10006, 1: 9994})


In [7]:
print(len(train))
print(len(validate))
print(len(test))

print(len(train_texts))
print(len(valid_texts))
print(len(test_texts))

60000
20000
20000
60000
20000
20000


In [8]:
# installing transformers since BERT resembles transformer encoder
! pip install transformers

     |████████████████████████████████| 573kB 1.4MB/s 
     |████████████████████████████████| 3.7MB 6.3MB/s 
     |████████████████████████████████| 870kB 17.2MB/s 
     |████████████████████████████████| 1.0MB 14.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=d1c15b4f9a1ca1f0c209207184a5ba33c2cc171ab63be436ae770712e525f822
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [0]:
from transformers import *

In [0]:
# define a function for data preparation
def data_prepare(file_path, lab2ind, tokenizer, max_len = 32, mode = 'train'):
    '''
    file_path: the path to input file. 
                In train mode, the input must be a tsv file that includes two columns where the first is text, and second column is label.
                The first row must be header of columns.

                In predict mode, the input must be a tsv file that includes only one column where the first is text.
                The first row must be header of column.

    lab2ind: dictionary of label classes
    tokenizer: BERT tokenizer
    max_len: maximal length of input sequence
    mode: train or predict
    '''
    # if we are in train mode, we will load two columns (i.e., text and label).
    if mode == 'train':
        # Use pandas to load dataset
        df = pd.read_csv(file_path, delimiter=',',header=0, names=['content','label'])
        print("Data size ", df.shape)
        labels = df.label.values
        
        # Create sentence and label lists
        labels = [i for i in labels] 
        print("Label is ", labels[0])
        
        # Convert data into torch tensors
        labels = torch.tensor(labels)

    # if we are in predict mode, we will load one column (i.e., text).
    elif mode == 'predict':
        df = pd.read_csv(file_path, delimiter=',',header=0, names=['content'])
        print("Data size ", df.shape)
        # create placeholder
        labels = []
    else:
        print("the type of mode should be either 'train' or 'predict'. ")
        return
        
    # Create sentence and label lists
    content = df.content.values

    # We need to add a special token at the beginning for BERT to work properly.
    content = ["[CLS] " + text for text in content]

    # Import the BERT tokenizer, used to convert our text into tokens that correspond to BERT's vocabulary.
    tokenized_texts = [tokenizer.tokenize(text) for text in content]

    
    # if the sequence is longer the maximal length, we truncate it to the pre-defined maximal length
    tokenized_texts = [ text[:max_len+1] for text in tokenized_texts]

    # We also need to add a special token at the end.
    tokenized_texts = [ text+['[SEP]'] for text in tokenized_texts]
    print ("Tokenize the first sentence:\n",tokenized_texts[0])
    
    # Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    print ("Index numbers of the first sentence:\n",input_ids[0])

    # Pad our input seqeunce to the fixed length (i.e., max_len) with index of [PAD] token
    pad_ind = tokenizer.convert_tokens_to_ids(['[PAD]'])[0]
    input_ids = pad_sequences(input_ids, maxlen=max_len+2, dtype="long", truncating="post", padding="post", value=pad_ind)
    print ("Index numbers of the first sentence after padding:\n",input_ids[0])

    # Create attention masks
    attention_masks = []

    # Create a mask of 1s for each token followed by 0s for pad tokens
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # Convert all of our data into torch tensors, the required datatype for our model
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, labels, masks

In [0]:
# build a binary classifier, 0 is GTP-2 generated, 1 is webtext
# lab2ind = {'GTP-2': 0, 'webtext': 1}

# tokenizer from pre-trained BERT model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)

In [18]:
# Use defined funtion to extract data
train_inputs, train_labels, train_masks = data_prepare(path+"subset_100k_train.csv", lab2ind,tokenizer)
validation_inputs, validation_labels, validation_masks = data_prepare(path+"subset_100k_valid.csv", lab2ind,tokenizer)

Data size  (60000, 2)
Label is  0
Tokenize the first sentence:
 ['[CLS]', 'mumbai', ':', 'india', "'", 's', 'star', 'professional', 'boxer', 'and', 'olympic', 'bronze', 'medalist', 'vi', '##je', '##nder', 'singh', 'will', 'clash', 'with', 'china', "'", 's', 'undefeated', ',', 'left', '-', 'handed', 'fighter', 'zu', '##lp', '##ika', '##r', '[SEP]']
Index numbers of the first sentence:
 [101, 8955, 1024, 2634, 1005, 1055, 2732, 2658, 10423, 1998, 4386, 4421, 12968, 6819, 6460, 11563, 5960, 2097, 13249, 2007, 2859, 1005, 1055, 15188, 1010, 2187, 1011, 4375, 4959, 16950, 14277, 7556, 2099, 102]
Index numbers of the first sentence after padding:
 [  101  8955  1024  2634  1005  1055  2732  2658 10423  1998  4386  4421
 12968  6819  6460 11563  5960  2097 13249  2007  2859  1005  1055 15188
  1010  2187  1011  4375  4959 16950 14277  7556  2099   102]
Data size  (20000, 2)
Label is  1
Tokenize the first sentence:
 ['[CLS]', 'a', 'recent', 'study', 'has', 'found', 'that', 'children', 'are', '

In [0]:
batch_size = 32
# We'll take training samples in random order in each epoch. 
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_dataloader = DataLoader(train_data, 
                              sampler = RandomSampler(train_data), # Select batches randomly
                              batch_size=batch_size)

# We'll just read validation set sequentially.
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_dataloader = DataLoader(validation_data, 
                                   sampler = SequentialSampler(validation_data), # Pull out batches sequentially.
                                   batch_size=batch_size)

In [20]:
model_path = "bert-base-uncased"

bert_model = BertModel.from_pretrained(model_path, output_hidden_states=True, output_attentions=True).to(device)

In [0]:
# explore the BERT base model

dataiter = iter(train_dataloader)
batch = dataiter.next()
# Add batch to GPU
batch = tuple(t.to(device) for t in batch)
# Unpack the inputs from our dataloader
input_ids, input_mask, labels = batch

In [0]:
last_hidden_state, pooler_output, hidden_states, attentions = bert_model(input_ids, attention_mask = input_mask)

In [23]:
# sequence of hidden-states at the output of the last layer.
last_hidden_state.shape # [batch size, seq length, hidden size]

torch.Size([32, 34, 768])

In [25]:
# pooler_output: last layer hidden-state of the first token of the sequence (classification token, [CLS]).
pooler_output.shape # [batch size, hidden size]

torch.Size([32, 768])

In [27]:
# (one for the output of each layer + the output of the embeddings) of shape [batch_size, sequence_length, hidden_size]: Hidden-states of the model at the output of each layer plus the initial embedding outputs.
print(len(hidden_states))

13


In [28]:
for i, item in enumerate(hidden_states):
  print("layer " + str(i), item.shape) # [batch size, sequence length, hidden size]

layer 0 torch.Size([32, 34, 768])
layer 1 torch.Size([32, 34, 768])
layer 2 torch.Size([32, 34, 768])
layer 3 torch.Size([32, 34, 768])
layer 4 torch.Size([32, 34, 768])
layer 5 torch.Size([32, 34, 768])
layer 6 torch.Size([32, 34, 768])
layer 7 torch.Size([32, 34, 768])
layer 8 torch.Size([32, 34, 768])
layer 9 torch.Size([32, 34, 768])
layer 10 torch.Size([32, 34, 768])
layer 11 torch.Size([32, 34, 768])
layer 12 torch.Size([32, 34, 768])


In [29]:
# Attentions weights after the attention softmax, used to compute the weighted average in the self-attention heads.
print(len(attentions))

12


In [30]:
for i, item in enumerate(attentions):
  print("layer " + str(i), item.shape) # [batch size, num_heads, sequence length, hidden size]

layer 0 torch.Size([32, 12, 34, 34])
layer 1 torch.Size([32, 12, 34, 34])
layer 2 torch.Size([32, 12, 34, 34])
layer 3 torch.Size([32, 12, 34, 34])
layer 4 torch.Size([32, 12, 34, 34])
layer 5 torch.Size([32, 12, 34, 34])
layer 6 torch.Size([32, 12, 34, 34])
layer 7 torch.Size([32, 12, 34, 34])
layer 8 torch.Size([32, 12, 34, 34])
layer 9 torch.Size([32, 12, 34, 34])
layer 10 torch.Size([32, 12, 34, 34])
layer 11 torch.Size([32, 12, 34, 34])


In [0]:
fc = nn.Linear(768, 2).to(device)

In [0]:
fc_output = fc(pooler_output)

In [33]:
# We use nn.CrossEntropyLoss() as our loss function. 
criterion = nn.CrossEntropyLoss()
criterion(fc_output, labels)

tensor(0.6918, device='cuda:0', grad_fn=<NllLossBackward>)

In [0]:
class Bert_cls(nn.Module):
    def __init__(self, lab2ind, model_path, hidden_size):
        super(Bert_cls, self).__init__()
        self.model_path = model_path
        self.hidden_size = hidden_size
        self.bert_model = BertModel.from_pretrained(model_path, output_hidden_states=True, output_attentions=True)
        self.label_num = len(lab2ind)
        self.fc = nn.Linear(self.hidden_size, self.label_num)
    def forward(self, bert_ids, bert_mask):
        last_hidden_state, pooler_output, hidden_states, attentions = self.bert_model(input_ids=bert_ids)
        fc_output = self.fc(pooler_output)
        return fc_output, attentions

In [0]:
bert_model = Bert_cls(lab2ind, 'bert-base-uncased', 768).to(device)

In [36]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(bert_model):,} trainable parameters')

The model has 109,483,778 trainable parameters


In [0]:
# Parameters:
lr = 2e-5
max_grad_norm = 1.0
epochs = 10
warmup_proportion = 0.1
num_training_steps  = len(train_dataloader) * epochs
num_warmup_steps = num_training_steps * warmup_proportion

### In Transformers, optimizer and schedules are instantiated like this:
# Note: AdamW is a class from the huggingface library
# the 'W' stands for 'Weight Decay"
optimizer = AdamW(bert_model.parameters(), lr=lr, correct_bias=False)
# schedules
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)  # PyTorch scheduler

# We use nn.CrossEntropyLoss() as our loss function. 
criterion = nn.CrossEntropyLoss()

In [0]:
def train(model, iterator, optimizer, scheduler, criterion):
    
    model.train()
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        input_ids, input_mask, labels = batch

        outputs,_ = model(input_ids, input_mask)

        loss = criterion(outputs, labels)
        # delete used variables to free GPU memory
        del batch, input_ids, input_mask, labels
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)  # Gradient clipping is not in AdamW anymore
        optimizer.step()
        scheduler.step()
        epoch_loss += loss.cpu().item()
        optimizer.zero_grad()
    
    # free GPU memory
    if device == 'cuda':
        torch.cuda.empty_cache()

    return epoch_loss / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    all_pred=[]
    all_label = []
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            # Add batch to GPU
            batch = tuple(t.to(device) for t in batch)
            # Unpack the inputs from our dataloader
            input_ids, input_mask, labels = batch

            outputs,_ = model(input_ids, input_mask)
            
            loss = criterion(outputs, labels)

            # delete used variables to free GPU memory
            del batch, input_ids, input_mask
            epoch_loss += loss.cpu().item()

            # identify the predicted class for each example in the batch
            probabilities, predicted = torch.max(outputs.cpu().data, 1)
            # put all the true labels and predictions to two lists
            all_pred.extend(predicted)
            all_label.extend(labels.cpu())
    
    accuracy = accuracy_score(all_label, all_pred)
    f1score = f1_score(all_label, all_pred, average='macro') 
    return epoch_loss / len(iterator), accuracy, f1score

In [0]:
# Train the model
loss_list = []
acc_list = []

for epoch in trange(epochs, desc="Epoch"):
    train_loss = train(bert_model, train_dataloader, optimizer, scheduler, criterion)  
    val_loss, val_acc, val_f1 = evaluate(bert_model, validation_dataloader, criterion)

    # Create checkpoint at end of each epoch
    state = {
        'epoch': epoch,
        'state_dict': bert_model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'scheduler': scheduler.state_dict()
        }

    torch.save(state, "./drive/My Drive/Colab Notebooks/585_group/ckpt_BERT/BERT_"+str(epoch+1)+".pt")

    print('\n Epoch [{}/{}], Train Loss: {:.4f}, Validation Loss: {:.4f}, Validation Accuracy: {:.4f}, Validation F1: {:.4f}'.format(epoch+1, epochs, train_loss, val_loss, val_acc, val_f1))


Epoch:  10%|█         | 1/10 [14:56<2:14:32, 896.93s/it]


 Epoch [1/10], Train Loss: 0.0769, Validation Loss: 0.2810, Validation Accuracy: 0.9142, Validation F1: 0.9139



Epoch:  20%|██        | 2/10 [29:54<1:59:36, 897.09s/it]


 Epoch [2/10], Train Loss: 0.0750, Validation Loss: 0.4139, Validation Accuracy: 0.9087, Validation F1: 0.9082



Epoch:  30%|███       | 3/10 [44:47<1:44:32, 896.03s/it]


 Epoch [3/10], Train Loss: 0.0457, Validation Loss: 0.3915, Validation Accuracy: 0.9178, Validation F1: 0.9176
